<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_03_4_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 3: Large Language Models**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 3 Material

* Part 3.1: Foundation Models [[Video]]() [[Notebook]](t81_559_class_03_1_llm.ipynb)
* Part 3.2: Text Generation [[Video]]() [[Notebook]](t81_559_class_03_2_text_gen.ipynb)
* Part 3.3: Text Summarization [[Video]]() [[Notebook]](t81_559_class_03_3_text_summary.ipynb)
* **Part 3.4: LLM Writes a Book** [[Video]]() [[Notebook]](t81_559_class_03_4_book.ipynb)
* Part 3.5: Small Large Language Models [[Video]]() [[Notebook]](t81_559_class_03_5_small_llm.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai pypdf pdfkit
    !apt-get install wkhtmltopdf

Note: using Google CoLab
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.6-2).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


# 3.4: LLM Writes a Book

This section will demonstrate how to effectively utilize the LLM to write a book. Writing a book is a uniquely complex challenge that requires planning and execution. Unlike simpler tasks, drafting a full-length book is a journey that extends beyond a single session with an LLM due to its extensive length and depth. However, with a systematic approach, you can harness the capabilities of an LLM to create a comprehensive manuscript, empowering you to conquer this challenge.

This section presents a method to craft a book using an LLM, which involves breaking down the book creation process into manageable segments. Our journey begins with a foundational step: selecting a subject for the book. Once a subject is chosen, the process unfolds through a series of iterative interactions with the LLM, each building upon the last to gradually construct the complete book, igniting your passion for writing.

Initially, we prompt the LLM to generate a random title based on the given subject. This title sets the thematic tone for the book and guides the subsequent steps. Following the title, we request the LLM to create a random synopsis. This synopsis acts as a narrative backbone, offering a brief overview of the book’s content and direction.

With a synopsis, the next step is to generate a table of contents. This table outlines the main chapters and sections of the book, providing a structured framework that organizes the content logically and cohesively. It is from this framework that the book starts to take shape.
The next phase involves creating each chapter. We engage in a focused session with the LLM for every chapter outlined in the table of contents. During each session, we provide the LLM with the synopsis and the complete table of contents to ensure consistency and continuity throughout the book. This iterative process allows each chapter to be crafted with attention to detail, ensuring it aligns with the previously established narrative and thematic elements.

By methodically iterating through these steps, we harness the capabilities of an LLM to transform a simple idea into a detailed and well-structured book. This chapter will guide you through each stage, providing practical advice on collaborating with an LLM to achieve more complex tasks.

We begin by accessing a large language model with a temperature of 0.7. We use a higher temperature to encourage creativity.

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain import OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown

#MODEL = 'gpt-4-turbo'
MODEL = 'gpt-3.5-turbo'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.7,
        n=1
    )


We create a simple utility function to query the LLM with a custom system prompt. The system prompt tells the LLM that it is assisting in creating a book.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

def query_llm(prompt):
  messages = [
      SystemMessage(
          content="You are assistant helping to write a book."
      ),
      HumanMessage(
          content=prompt
      ),
  ]

  output = llm.invoke(messages)
  return output.content


## Generate Title, Synopsis and Table of Contents

For this book, we allow the user to specify the subject specified by the SUBJECT variable. We then request the LLM to generate a random title based on this subject. It is essential that the prompt request that the LLM only the; LLMs often like to prefix with text such as "Here is a random title."

In [ ]:
SUBJECT = "historic romance novel"

title = query_llm(f"""
Give me a random title for a book on the subject '{SUBJECT}'.
Return only the title, no additional text.
""").strip(" '\"")
print(title)

Echoes of Eternity


Now that we have a title, we can request a random synopsis of the book.

In [ ]:
synopsis = query_llm(f"""
Give me a synopsis for a book of the title '{SUBJECT}' for a book on the subject '{SUBJECT}'.
Return only the synopsis, no additional text.
""").strip(" '\"")
print(synopsis)

In the grand halls of 18th century England, Lady Elizabeth and the dashing Duke of Pembroke find themselves entangled in a forbidden romance that threatens to unravel the fabric of society. As they navigate treacherous political alliances and societal expectations, their love deepens, igniting a passion that cannot be denied. But when a long-buried secret emerges, their bond is put to the ultimate test. Will they choose duty over desire, or will love conquer all in this sweeping historic romance novel?


Next, we generate the table of contents. For this generation, we provide all previous information. We also request a particular format for the table of contents. You may notice that I ask for the chapter numbers, even though they are an increasing numeric count and could easily be derived. This process works better because the LLM wants to provide the chapter numbers, and attempts to suppress chapter numbers are often inconsistent. It is easier to allow the LLM to generate chapter numbers but control where it generates them so that I can consistently remove them later.

In [ ]:
toc = query_llm(f"""
Give me a table of contents for a book of the title '{title}' for a book on
the subject '{SUBJECT}' the book synopsis is '{synopsis}'.
Return the table of contents as a list of chapter titles.
Separate the chapter number and chapter title with a pipe character '|'.
Return only the chapter names, no additional text.
""").strip(" '\"")
print(toc)

1|The Grand Ball
2|Forbidden Encounters
3|Intrigues of the Court
4|Whispers of Scandal
5|Passion Unleashed
6|A Love Tested
7|Revelations in the Moonlight
8|Betrayal and Redemption
9|The Choice of the Heart
10|Echoes of Eternity


We must now parse the table of contents and remove the pipes and chapter numbers.

In [ ]:
# Split the string into lines
lines = toc.splitlines()

# Extract titles using list comprehension
toc2 = [line.split('|')[1].strip() for line in lines if line]

# Print the list of titles
print(toc2)

['The Grand Ball', 'Forbidden Encounters', 'Intrigues of the Court', 'Whispers of Scandal', 'Passion Unleashed', 'A Love Tested', 'Revelations in the Moonlight', 'Betrayal and Redemption', 'The Choice of the Heart', 'Echoes of Eternity']


## Generate the Chapters of the Book

Next, we create a function capable of producing the text that makes up a chapter. To ensure that the function has enough context to generate each chapter, we provide the synopsis, the table of contents, and the chapter number. To test this code, we request that it develop a single chapter.

In [ ]:
def render_chapter(num, chapter_title, title, subject, synopsis, toc):
  txt = query_llm(f"""
  Write Chapter {num}, titled "{chapter_title}" for a book of the title '{title}' for a book on
  the subject '{subject}' the book synopsis is '{synopsis}' the table of contents is '{toc}'.
  Give me only the chapter text, no chapter heading, no chapter title, number, no additional text.
  """).strip(" '\"")
  return txt

txt = render_chapter(1, toc2[0], title, SUBJECT, synopsis, toc)
print(txt)

The Grand Ball

The grand ballroom glittered with the soft glow of candlelight, casting a warm and inviting ambiance over the opulent affair. Lady Elizabeth glided across the polished marble floor, her emerald gown trailing behind her in a mesmerizing swirl of silk and lace. Her heart beat with anticipation as she searched the crowded room for a familiar face, her eyes finally landing on the dashing figure of the Duke of Pembroke.

As their gazes met, a spark ignited between them, a silent understanding passing between their souls. The forbidden attraction that had been simmering beneath the surface flared to life, threatening to consume them both in its fiery embrace. But in a society bound by strict rules and expectations, their love was a dangerous game, a scandal waiting to happen.

Despite the risks, Lady Elizabeth found herself drawn to the Duke like a moth to a flame. His charm and wit captivated her, his touch sending shivers of delight down her spine. And as they danced togeth

We can now generate the entire book in Markdown, which allows some formatting. We begin by rendering the title and synopsis, the table of contents, and each chapter.

In [ ]:
book = ""

# Render the title and synopsis
book += f"# {title}\n"
book += f"{synopsis}\n"

# Render the toc
book += f"\n## Table of Contents\n\n"
num = 1
for chapter_title in toc2:
  book += f"{num}. {chapter_title}\n"
  num += 1

# Render the book
chapter = 1
for chapter_title in toc2:
  print(f"Rendering chapter {chapter}/{len(toc2)}: {chapter_title}")
  txt = render_chapter(chapter, chapter_title, title, SUBJECT, synopsis, toc)
  book += f"\n\n## Chapter {chapter}: {chapter_title}\n"
  book += f"{txt}\n"
  chapter += 1

Rendering chapter 1/10: The Grand Ball
Rendering chapter 2/10: Forbidden Encounters
Rendering chapter 3/10: Intrigues of the Court
Rendering chapter 4/10: Whispers of Scandal
Rendering chapter 5/10: Passion Unleashed
Rendering chapter 6/10: A Love Tested
Rendering chapter 7/10: Revelations in the Moonlight
Rendering chapter 8/10: Betrayal and Redemption
Rendering chapter 9/10: The Choice of the Heart
Rendering chapter 10/10: Echoes of Eternity


## Generate a PDF of the Book

Now that we have generated the book, we have saved it as a PDF.

In [ ]:
import markdown
import pdfkit

# Convert Markdown to HTML
html = markdown.markdown(book)

# Convert HTML to PDF
pdfkit.from_string(html, 'output.pdf')

True

We can now download the generated book.

In [ ]:
from google.colab import files
files.download("output.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>